In [ ]:
import torch
torch.cuda.is_available()
!pip install hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 42.4 MB/s  0:00:00


In [ ]:
!python /workspace/Hypertension-AI/src/prepare_hypertension_llm_data_2.py \
  --data-path /workspace/Hypertension-AI/data/hypertension_dataset.csv \
  --output-dir /workspace/Hypertension-AI/data/llm \
  --base-model meta-llama/Llama-3.1-8B-Instruct

tokenizer_config.json: 100%|███████████████| 55.4k/55.4k [00:00<00:00, 65.5MB/s]
tokenizer.json: 100%|██████████████████████| 9.09M/9.09M [00:00<00:00, 20.2MB/s]
special_tokens_map.json: 100%|█████████████████| 296/296 [00:00<00:00, 1.16MB/s]
{
  "train": 1607,
  "valid": 179,
  "test": 199,
  "stress_thresholds": {
    "low_max": 6.0,
    "moderate_max": 10.0
  },
  "prompt_style": "chat_template_llama"
}


In [ ]:
!python /workspace/Hypertension-AI/src/fine_tune_hypertension_llm_2.py \
  --train-file /workspace/Hypertension-AI/data/llm/train.jsonl \
  --valid-file /workspace/Hypertension-AI/data/llm/valid.jsonl \
  --test-file /workspace/Hypertension-AI/data/llm/test.jsonl \
  --output-dir /workspace/Hypertension-AI/artifacts/llm \
  --base-model meta-llama/Llama-3.1-8B-Instruct \
  --num-epochs 3 \
  --batch-size 2 \
  --gradient-accumulation 8 \
  --learning-rate 1e-4 \
  --max-length 512 \
  --max-new-tokens 24 \
  --load-in-4bit

Generating train split: 1607 examples [00:00, 28908.49 examples/s]
Generating validation split: 179 examples [00:00, 11177.98 examples/s]
config.json: 100%|█████████████████████████████| 855/855 [00:00<00:00, 3.08MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
model.safetensors.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 62.3MB/s]
model-00001-of-00004.safetensors: 100%|█████| 4.98G/4.98G [00:25<00:00, 196MB/s]
model-00002-of-00004.safetensors: 100%|█████| 5.00G/5.00G [00:22<00:00, 223MB/s]
model-00003-of-00004.safetensors: 100%|█████| 4.92G/4.92G [00:27<00:00, 181MB/s]
model-00004-of-00004.safetensors: 100%|█████| 1.17G/1.17G [00:08<00:00, 146MB/s]
generation_config.json: 100%|███████████████████| 184/184 [00:00<00:00, 982kB/s]
trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196
  0%|                                                   | 0/303 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=Fals

In [ ]:
!python /workspace/Hypertension-AI/src/fine_tune_hypertension_llm_2.py \
  --eval-only \
  --test-file /workspace/Hypertension-AI/data/llm/test.jsonl \
  --base-model meta-llama/Llama-3.1-8B-Instruct \
  --adapter-dir /workspace/Hypertension-AI/artifacts/llm/lora_adapter \
  --max-new-tokens 24

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:46<00:00, 11.53s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
{
  "adapter_dir": "/workspace/Hypertension-AI/artifacts/llm/lora_adapter",
  "test_metrics": {
    "num_samples": 199,
    "hypertension_accuracy": 0.9798994974874372,
    "stress_accuracy": 1.0,
    "joint_exact_match": 0.9798994974874372
  }
}


In [ ]:
!python /workspace/Hypertension-AI/src/evaluate_llm_detailed.py \
  --base-model meta-llama/Llama-3.1-8B-Instruct \
  --adapter-dir /workspace/Hypertension-AI/artifacts/llm/lora_adapter \
  --data-file /workspace/Hypertension-AI/data/llm/test.jsonl \
  --max-new-tokens 24

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:03<00:00,  1.28it/s]
Generating data split: 199 examples [00:00, 4627.24 examples/s]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
{
  "n_samples": 199,
  "hypertension": {
    "tp": 101,
    "tn": 94,
    "fp": 2,
    "fn": 2,
    "accuracy": 0.9798994974874372,
    "precision": 0.9805825242718447,
    "recall": 0.9805825242718447,
    "specificity": 0.9791666666666666,
    "f1": 0.9805825242718447
  },
  "stress": {
    "accuracy": 1.0
  }
}
